In [ ]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from copy import deepcopy
from tqdm import tqdm
import random
from random import shuffle

In [ ]:
import cv2
import numpy as np
import random
import os
import re
from tqdm import tqdm
from copy import deepcopy
import sys

import cv2
import numpy as np
from tqdm import tqdm
from multiprocessing import Process, Manager


In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

def live_plot(data, figsize=(16,8), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.imshow(convert_to_rgb(data.astype(int)))
    plt.title(title)
    plt.show()

## 读取数据

In [ ]:
def convert_to_rgb(img):
    img = deepcopy(img)
    b,g,r = cv2.split(img)  
    img = cv2.merge([r,g,b])
    return img

In [ ]:
background = cv2.imread('../../base_data/background.jpeg')
hedge = cv2.imread('../../base_data/hedge.jpeg')

## 生成视频代码

In [ ]:
def generate_video(frame_dir=None, video_dir=None, video_name='video.avi'):

    image_num = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
    image_name_list = [f'{i}.jpg' for i in range(image_num)]
    frame = cv2.imread(os.path.join(frame_dir, image_name_list[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(f'{video_dir}/{video_name}', 0, 20, (width, height))

    # video = cv2.VideoWriter(f'{self.video_dir}/{video_name}', 0, 20, (width, height))

    for image in image_name_list:
        video.write(cv2.imread(os.path.join(frame_dir, image)))

    cv2.destroyAllWindows()
    video.release()

## 显示图片

In [ ]:
plt.imshow(convert_to_rgb(background))
plt.show()
plt.imshow(convert_to_rgb(hedge))
plt.show()

In [ ]:
background.shape, hedge.shape

## 定义参数

In [ ]:
speed_dict = {
    "background": 1,
    "hedge_1": 2,
    "hedge_2": 3,
    "black_bar": 4
}

crop_height = 900
crop_width = 1500
slide_width = 800
camera_width = 250

crop_width_extend = crop_width + 1000

size_1 = camera_width * 2
size_2 = slide_width

dimension = 3
black_bar_width = 10

background_prob = 0.7
noise_average_size = 10

vertical_movement = [0]
horizontal_movement = [1, 2]

In [ ]:
background_crop = deepcopy(background[-crop_height:, :crop_width, :])
live_plot(background_crop)

In [ ]:
hedge_crop = deepcopy(hedge[-crop_height:, :crop_width, :])
live_plot(hedge_crop)

## 生成噪声图

In [ ]:
# def generate_black_bar_image():
#     black_bar_image = np.zeros((crop_height, crop_width_extend, dimension))
#     for i in range(150, crop_width_extend, 200):
#         black_bar_image[:, i - black_bar_width:i + black_bar_width, :] = 1
#     return black_bar_image

# def generate_random_hole():
#     random_hole = np.random.random_sample((crop_height + noise_average_size,
#                                            crop_width_extend + noise_average_size))
#     random_hole = np.round(random_hole - (background_prob / 2))
#     group_random_hole = np.zeros((crop_height, crop_width))
#     for i in range(group_random_hole.shape[0]):
#         for j in range(group_random_hole.shape[1]):
#             group_random_hole[i, j] = random_hole[i:i + noise_average_size, j:j + noise_average_size].mean()
#     group_random_hole = np.round(group_random_hole / group_random_hole.max())
#     group_random_hole = np.array([[[j] * 3 for j in i] for i in group_random_hole])
#     return group_random_hole.astype(int)

In [ ]:
# random_hole_1 = generate_random_hole()
# live_plot(random_hole_1 * 255)

In [ ]:
# random_hole_2 = generate_random_hole()
# live_plot(random_hole_2 * 255)

In [ ]:
# black_bar_image = generate_black_bar_image()
# live_plot(((1 - black_bar_image)*255).astype(int))

In [ ]:
# # random_hole_1 = generate_random_hole()
# # random_hole_2 = generate_random_hole()
# # black_bar_image = generate_black_bar_image()
# verti_pos = int(background_crop.shape[0]/2)
# hori_pos = 0
# for i in tqdm(range(10000), ncols=70):
#     try:
#         upper, lower = verti_pos + camera_width, verti_pos - camera_width
#         background_pos = int(speed_dict['background'] * hori_pos)
#         hedge_1_pos = int(speed_dict['hedge_1'] * hori_pos)
#         hedge_2_pos = int(speed_dict['hedge_2'] * hori_pos)
#         black_bar_pos = int(speed_dict['black_bar'] * hori_pos)

#         background_crop_temp = background_crop[int(lower):int(upper), background_pos:background_pos + slide_width, :]
#         hedge_1_image_temp = hedge_crop[int(lower):int(upper), hedge_1_pos:hedge_1_pos + slide_width, :]
#         random_hole_1_temp = random_hole_1[int(lower):int(upper), hedge_1_pos:hedge_1_pos + slide_width, :]

#         hedge_2_image_temp = hedge_crop[int(lower):int(upper), hedge_2_pos:hedge_2_pos + slide_width, :]
#         random_hole_2_temp = random_hole_2[int(lower):int(upper), hedge_2_pos:hedge_2_pos + slide_width, :]

#         black_bar_image_temp = black_bar_image[int(lower):int(upper), black_bar_pos:black_bar_pos + slide_width, :]

#         output_image = background_crop_temp * (1 - random_hole_1_temp) + hedge_1_image_temp * random_hole_1_temp
#         output_image = output_image * (1 - random_hole_2_temp) + hedge_2_image_temp * random_hole_2_temp
#         output_image = output_image * (1 - black_bar_image_temp) + black_bar_image_temp
#         output_image = output_image.astype(int)
#         cv2.imwrite(f"../scene_data/{i}_{verti_pos}_{hori_pos}.jpg", output_image)
#         live_plot(output_image)
#         hori_pos += random.choice(horizontal_movement)
#         verti_pos += random.choice(vertical_movement)
#     except Exception as e:
#         print(e)
#         break

## 基于已知位移的恢复算法

In [ ]:
def rgb_mean(image):
    return tuple([np.mean([np.mean(image[:, :, i])]) for i in range(2, -1, -1)])

def initialise_image(image):
    reshape_size = image.shape[0] * image.shape[1]
    a = np.zeros((dimension, reshape_size))
    for i in range(3):
        a[i] = np.array(image[:, :, i]).reshape(1, reshape_size)
    return a

def random_select():
    return random.randint(0, 250), random.randint(0, 750)

def random_radius():
    return random.randint(10, 35)

def mahalanobis(x, mean, cov):
    return np.sqrt(np.linalg.inv(cov).dot(np.transpose(x - mean)).dot((x - mean)))

def cal_ratio_prob(x):
    return 1 - (x/x.sum())

def ratio_soft_max(self, x):
    return ratio_prob(soft_max(x))

def soft_max(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
# scene_dict

In [ ]:
def calculate_background_prob(pixel):
    hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
    top_distance = mahalanobis(pixel, background_mean, background_cov)
    ratio_prob = ratio_prob(np.array([top_distance, hedge_distance]))[0]
    return ratio_prob

def breakdown_sequence(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

def sub_cal_prob(data_set, prob_dict):
    for pixel, key in data_set:
        prob_dict[key] = calculate_background_prob(pixel)

def detect_background_pixel(frame, threshold=0.5):
    manager = Manager()  # create only 1 mgr
    prob_dict = manager.dict()

    data_set_list = []
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            data_set_list.append((pixel, (i, j)))

    data_set_list = breakdown_sequence(data_set_list, 16)

    process_list = []
    for data_set in data_set_list:
        p = Process(target=sub_cal_prob, args=(data_set, prob_dict))
        p.start()
        process_list.append(p)

    for p in process_list:
        p.join()

    frame_background_prob = np.zeros(frame.shape)
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            ratio_prob = prob_dict[(i, j)]
            # ratio_prob = ratio_prob if ratio_prob > threshold else 0
            frame_background_prob[i, j, :] = ratio_prob

    return frame_background_prob

def calculate_diff(last_frame, current_frame, compare_range=3):
    current_frame_crop = deepcopy(current_frame[compare_range:-compare_range, compare_range:-compare_range, :])
    current_frame_crop_prob = detect_background_pixel(current_frame_crop)
    # current_frame_crop = np.round(current_frame_crop)

    diff_dict = {}
    shape_0, shape_1 = current_frame_crop.shape[:2]
    for i in tqdm(range(compare_range*2), ncols=70):
        for j in range(compare_range*2):
            last_frame_crop = deepcopy(last_frame[i:shape_0+i, j:shape_1+j, :])
            last_frame_crop_prob = detect_background_pixel(last_frame_crop)
            # last_frame_crop_prob = np.round(last_frame_crop_prob)

            cross_prob = (current_frame_crop_prob + last_frame_crop_prob)/2
            current_diff = np.sum((current_frame_crop - last_frame_crop) * cross_prob)/np.sum(cross_prob)

            diff_dict[(i-compare_range, j-compare_range)] = current_diff

    print(min(diff_dict, key=diff_dict.get))
    return min(diff_dict, key=diff_dict.get)    

In [ ]:
# frame_dir = '../scene_data'
# count_img = np.zeros([size_1, size_2])
# track_img = np.zeros([size_1, size_2, dimension])


# hedge_mean = np.array(rgb_mean(hedge_crop))
# background_mean = np.array(rgb_mean(background_crop))
# hedge_cov = np.cov(initialise_image(hedge_crop))
# background_cov = np.cov(initialise_image(background_crop))

# scene_dict = {}
# for file_name in os.listdir(frame_dir):
#     if file_name[0] != '.':
#         num, verti_pos, hori_pos = file_name.replace('.jpg', '').split('_')
#         scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}

# num_of_scene = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
# test_image_dict = {}
# proba_dict = {}
# pixel_dict = {}

# threshold=0.5
# ratio_prob_image = np.zeros([size_1 + 0, size_2 + 100 + num_of_scene, 3])
# output_image = deepcopy(ratio_prob_image)
# visit_image = deepcopy(ratio_prob_image)

# offset = 150
# t_pos = 0
# last_frame = None
# count = 0

# for num in tqdm([random.choice(list(scene_dict.keys())) for _ in range(2000)], ncols=70):
#     file_name = scene_dict[num]['file_name']
#     verti_pos = scene_dict[num]['verti_pos'] - 450
#     hori_pos = scene_dict[num]['hori_pos']
#     frame = np.array(cv2.imread(f"{frame_dir}/{file_name}"))

#     visit_image_sum = np.sum(ratio_prob_image, axis=2)
#     rank_list = []
#     base_line = random.choice(list(range(visit_image_sum.shape[0]-2)))
    
#     for vertical_pos in range(visit_image_sum.shape[0]):
#         for horizontal_pos in range(visit_image_sum.shape[1]):
#             i = vertical_pos - verti_pos
#             j = horizontal_pos - hori_pos
            
#             if frame.shape[0] > i >= 0 and frame.shape[1] > j >= 0:
#                 rank_list.append([visit_image_sum[vertical_pos,horizontal_pos], (i,j)])
    
# #     shuffle(rank_list)
# #     rank_list = rank_list[:400000]
# #     if random.random() > 0.5:
#     number_of_points = 100000/np.log(np.sum(visit_image_sum))
#     number_of_points = 1000 if number_of_points == 0 else number_of_points
#     number_of_points = np.clip(number_of_points, 1000, 100000)
#     number_of_points = int(number_of_points)
    
#     rank_list = list(sorted(rank_list))[:number_of_points]
# #     else:
# #         rank_list = rank_list[:100000]
    
#     for _, pos in rank_list:
#         i, j = pos
# #         i = random.choice(range(frame.shape[0]))
# #         j = random.choice(range(frame.shape[1]))
        
#         vertical_pos = int(verti_pos + i)
#         horizontal_pos = int(hori_pos + j)
#         key = (vertical_pos, horizontal_pos)
#         pixel = frame[i][j]
#         hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
#         top_distance = mahalanobis(pixel, background_mean, background_cov)
#         ratio_prob = cal_ratio_prob(np.array([top_distance, hedge_distance]))[0]
#         proba_dict[key] = proba_dict.get(key, {})
        
#         ratio_prob = ratio_prob if ratio_prob > threshold else 0

#         if ratio_prob > 0:
#             temp_pixel = deepcopy(ratio_prob_image[vertical_pos, horizontal_pos, :])
#             temp_prob = proba_dict[key].get('ratio_prob', 0)

#             pixel_before = temp_pixel/proba_dict[key].get('ratio_prob', 0)
#             pixel_after = (temp_pixel + pixel * ratio_prob)/(proba_dict[key].get('ratio_prob', 0) + ratio_prob)

#             before_distance = mahalanobis(pixel_before, background_mean, background_cov)
#             after_distance = mahalanobis(pixel_after, background_mean, background_cov)
            
# #             if after_distance < before_distance or before_distance != before_distance:
#             proba_dict[key]['ratio_prob'] = proba_dict[key].get('ratio_prob', 0) + ratio_prob
#             ratio_prob_image[vertical_pos, horizontal_pos, :] += pixel * ratio_prob
#             visit_image[vertical_pos, horizontal_pos, :] += 1
        
        
#     output_image = deepcopy(ratio_prob_image)
#     for i, j in proba_dict.keys():
#         if proba_dict[(i, j)].get('ratio_prob', 0) > 0:
#             output_image[i, j] = output_image[i, j] / proba_dict[(i, j)]['ratio_prob']
#     cv2.imwrite(f"../recover_data/{count}.jpg", output_image)
#     live_plot(output_image)
#     count += 1
    
#     print(base_line)
#     print(number_of_points)
    
# generate_video(frame_dir='../recover_data', video_dir='../video_data', video_name=f'recover.avi')

In [ ]:
frame_dir = '../scene_data'
count_img = np.zeros([size_1, size_2])
track_img = np.zeros([size_1, size_2, dimension])


hedge_mean = np.array(rgb_mean(hedge_crop))
background_mean = np.array(rgb_mean(background_crop))
hedge_cov = np.cov(initialise_image(hedge_crop))
background_cov = np.cov(initialise_image(background_crop))

scene_dict = {}
for file_name in os.listdir(frame_dir):
    if file_name[0] != '.':
        num, verti_pos, hori_pos = file_name.replace('.jpg', '').split('_')
        scene_dict[int(num)] = {'file_name': file_name, 'num':num, 'verti_pos':int(verti_pos), 'hori_pos':int(hori_pos)}

num_of_scene = len([i for i in os.listdir(frame_dir) if '.jpg' in i])
test_image_dict = {}
proba_dict = {}
pixel_dict = {}

threshold=0.5
ratio_prob_image = np.zeros([size_1 + 0, size_2 + 100 + num_of_scene, 3])
output_image = deepcopy(ratio_prob_image)
visit_image = deepcopy(ratio_prob_image)

offset = 150
t_pos = 0
last_frame = None
count = 0

for num in tqdm([random.choice(list(scene_dict.keys())) for _ in range(2000)], ncols=70):
    file_name = scene_dict[num]['file_name']
    verti_pos = scene_dict[num]['verti_pos'] - 450
    hori_pos = scene_dict[num]['hori_pos']
    frame = np.array(cv2.imread(f"{frame_dir}/{file_name}"))

    visit_image_sum = np.sum(ratio_prob_image, axis=2)
    rank_list = []
    base_line = random.choice(list(range(visit_image_sum.shape[0]-2)))
    
    for vertical_pos in range(visit_image_sum.shape[0]):
        for horizontal_pos in range(visit_image_sum.shape[1]):
            i = vertical_pos - verti_pos
            j = horizontal_pos - hori_pos
            
            if frame.shape[0] > i >= 0 and frame.shape[1] > j >= 0:
                rank_list.append([visit_image_sum[vertical_pos,horizontal_pos], (i,j)])
    
#     shuffle(rank_list)
#     rank_list = rank_list[:400000]
#     if random.random() > 0.5:
    number_of_points = 400000/np.log(np.sum(visit_image_sum))
    number_of_points = 1000 if number_of_points == 0 else number_of_points
    number_of_points = np.clip(number_of_points, 1000, 400000)
    number_of_points = int(number_of_points)
    
    rank_list = list(sorted(rank_list))[:number_of_points]
#     else:
#         rank_list = rank_list[:100000]
    
    for _, pos in rank_list:
        i, j = pos
#         i = random.choice(range(frame.shape[0]))
#         j = random.choice(range(frame.shape[1]))
        
        vertical_pos = int(verti_pos + i)
        horizontal_pos = int(hori_pos + j)
        key = (vertical_pos, horizontal_pos)
        pixel = frame[i][j]
        hedge_distance = mahalanobis(pixel, hedge_mean, hedge_cov)
        top_distance = mahalanobis(pixel, background_mean, background_cov)
        ratio_prob = cal_ratio_prob(np.array([top_distance, hedge_distance]))[0]
        proba_dict[key] = proba_dict.get(key, {})
        
        ratio_prob = ratio_prob if ratio_prob > threshold else 0

        if ratio_prob > 0:
            temp_pixel = deepcopy(ratio_prob_image[vertical_pos, horizontal_pos, :])
            temp_prob = proba_dict[key].get('ratio_prob', 0)

            pixel_before = temp_pixel/proba_dict[key].get('ratio_prob', 0)
            pixel_after = (temp_pixel + pixel * ratio_prob)/(proba_dict[key].get('ratio_prob', 0) + ratio_prob)

            before_distance = mahalanobis(pixel_before, background_mean, background_cov)
            after_distance = mahalanobis(pixel_after, background_mean, background_cov)
            
            if after_distance < before_distance or before_distance != before_distance:
                proba_dict[key]['ratio_prob'] = proba_dict[key].get('ratio_prob', 0) + ratio_prob
                ratio_prob_image[vertical_pos, horizontal_pos, :] += pixel * ratio_prob
                visit_image[vertical_pos, horizontal_pos, :] += 1
        
        
    output_image = deepcopy(ratio_prob_image)
    for i, j in proba_dict.keys():
        if proba_dict[(i, j)].get('ratio_prob', 0) > 0:
            output_image[i, j] = output_image[i, j] / proba_dict[(i, j)]['ratio_prob']
    cv2.imwrite(f"../recover_data_1/{count}.jpg", output_image)
    live_plot(output_image)
    count += 1
    
    print(base_line)
    print(number_of_points)
    
generate_video(frame_dir='../recover_data_1', video_dir='../video_data_1', video_name=f'recover.avi')